In [1]:
import sys
print(sys.executable)

/home/maxime/dev/python/sndx/.venv/bin/python3.12


In [2]:
import os
import time
import asyncio
import sndx
from pathlib import Path
from datetime import timedelta

output_dir = (Path(os.getcwd()) / '..' / 'output').resolve()
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
profile = "sndx-profile-1"

with sndx.Sink() as sink:
    async with sndx.Scrapper(sink, profile) as scrapper:
        print("Doing stuff")
        await asyncio.sleep(3)
        print("Stuff finished")

Opening sink sndx-p1S94A
[Crawler-pPPezQ] Starting with profile [sndx-profile-1] and sink [sndx-p1S94A]...
Doing stuff
Stuff finished
[Crawler-pPPezQ] Terminating...
Closing sink sndx-p1S94A


In [2]:
sink = sndx.Sink()
sink.__enter__()
print(sink)

Sink(id='pOyRpg', module_id='536870916')


In [4]:
profile1 = "01"
agent = sndx.Agent(sink, profile1)
await agent.__aenter__()
print(agent)

In [25]:
video_url = "https://vod.catalogue-crc.org/enregistrement/la-nouvelle-droite.html"
await agent.scrap_recording(video_url)

Started recording from sndx-uhrPkA in /home/maxime/dev/python/sndx/output/LA_NOUVELLE_DROITE.mp3.


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.3.0 (GCC)
  configuration: --disable-static --prefix=/nix/store/eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee-ffmpeg-7.1.1 --target_os=linux --arch=x86_64 --pkg-config=pkg-config --enable-gpl --enable-version3 --disable-nonfree --disable-static --enable-shared --enable-pic --disable-thumb --disable-small --enable-runtime-cpudetect --disable-gray --enable-swscale-alpha --enable-hardcoded-tables --enable-safe-bitstream-reader --enable-pthreads --disable-w32threads --disable-os2threads --enable-network --enable-pixelutils --datadir=/nix/store/gv91xj7x2ll37vjpnja3gr1hgz9xshx8-ffmpeg-7.1.1-data/share/ffmpeg --enable-ffmpeg --enable-ffplay --enable-ffprobe --bindir=/nix/store/eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee-ffmpeg-7.1.1-bin/bin --enable-avcodec --enable-avdevice --enable-avfilter --enable-avformat --enable-avutil --enable-postproc --enable-swresample --enable-swscale --libdir=/nix/store/eeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

CancelledError: 

[out#0/mp3 @ 0x4c3300] video:0KiB audio:856KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.070722%
size=     857KiB time=00:00:36.51 bitrate= 192.2kbits/s speed=0.999x    
Exiting normally, received signal 2.


In [12]:
await agent.__aexit__(None, None, None)

## Helpers

In [129]:
metadata = await extract_metadata(page)

print(f"URL: {metadata.url}")
print(f"Category: {metadata.category}")
print(f"Title: {metadata.title}")
print(f"Subtitle: {metadata.subtitle}")
print(f"Code: {metadata.code}")
print(f"Date: {metadata.date}")
print(f"Place: {metadata.place}")
print(f"Authors: {metadata.authors}")
print(f"Duration: {metadata.duration}")

URL: https://vod.catalogue-crc.org/enregistrement/la-nouvelle-droite.html
Category: Les grands affrontements du siècle
Title: LA NOUVELLE DROITE
Subtitle: Alain de Benoist, surhomme indo-européen.
Code: AS 1
Date: 25 octobre 1979
Place: Paris
Authors: ['Abbé Georges de Nantes']
Duration: 1:01:43


In [120]:
elements = await page.xpath("//ul[@id='details']//dd")
(await get_text(elements[4])).strip()
[s.strip() for s in (await get_text(elements[3])).split("<br/>")]

['Abbé Georges de Nantes']

In [99]:
await text(first(page.xpath("//h2[preceding-sibling::*[3][self::h3]]")))


'Alain de Benoist, surhomme indo-européen.'

## Record
Cliquer sur "Audio bas débit" lance la lecture audio.

In [78]:
video_url = "https://vod.catalogue-crc.org/enregistrement/la-nouvelle-droite.html"
await goto_logged_in(page, video_url)

In [43]:
elements = await page.xpath("//span[contains(@class, 'vjs-duration-display')]")
text = await page.evaluate('(el) => el.textContent', elements[0])
duration = parse_duration(text.strip())
duration

datetime.timedelta(seconds=3703)

In [62]:
sink.__exit__(None, None, None)